# Translate music token sequence back into MIDI

### Install and import libraries

In [ ]:
!pip install note_seq

In [ ]:
!apt-get install -qq fluidsynth
!apt-get install -qq libfluidsynth-dev

In [ ]:
#!pip uninstall -y pyfluidsynth
!pip install pyfluidsynth

In [ ]:
!wget https://github.com/FluidSynth/fluidsynth/wiki/SoundFont

The following methods are modified from https://github.com/AI-Guru/MMM-JSB/

In [ ]:
import note_seq
import fluidsynth
NOTE_LENGTH_16TH_120BPM = 0.25 * 60 / 120
BAR_LENGTH_120BPM = 4.0 * 60 / 120

In [ ]:
def empty_note_sequence(qpm=120.0, total_time=0.0):
    note_sequence = note_seq.protobuf.music_pb2.NoteSequence()
    note_sequence.tempos.add().qpm = qpm
    note_sequence.ticks_per_quarter = note_seq.constants.STANDARD_PPQ
    note_sequence.total_time = total_time
    return note_sequence

In [ ]:
def token_sequence_to_note_sequence(token_sequence):

    if isinstance(token_sequence, str):
        token_sequence = token_sequence.split()

    note_sequence = empty_note_sequence()
    current_program = 1
    current_is_drum = False
    for token_index, token in enumerate(token_sequence):
        if token.startswith("INST"):
            current_bar_index = 0
            current_instrument = token.split("=")[-1]
        elif token == "BAR_START":
            current_time = current_bar_index * BAR_LENGTH_120BPM
            current_notes = {}
        elif token == "BAR_END":
            current_bar_index += 1
            pass
        elif token.startswith("NOTE_ON"):
            pitch = int(token.split("=")[-1])
            note = note_sequence.notes.add()
            note.start_time = current_time
            note.end_time = current_time + 4 * NOTE_LENGTH_16TH_120BPM
            note.pitch = pitch
            note.instrument = int(current_instrument)
            note.program = current_program
            note.velocity = 80
            note.is_drum = False
            current_notes[pitch] = note
        elif token.startswith("NOTE_OFF"):
            pitch = int(token.split("=")[-1])
            if pitch in current_notes:
                note = current_notes[pitch]
                note.end_time = current_time
        elif token.startswith("TIME_DELTA"):
            delta = float(token.split("=")[-1]) * NOTE_LENGTH_16TH_120BPM
            current_time += delta
        elif token == "[PAD]":
            pass
        else:
            assert False, token

    return note_sequence

In [ ]:
def render_token_sequence(token_sequence):
    note_sequence = token_sequence_to_note_sequence(token_sequence)
    note_seq.plot_sequence(note_sequence)
    #synth = note_seq.midi_synth.fluidsynth
    #note_seq.play_sequence(note_sequence, synth)
    #return note_sequence

### Process token sequences files

In [ ]:
def process_files():
    finput = open("input_data_valid.txt", "r")
    foutput = open("output_data.txt", "r")
    ftarget = open("target_data_valid.txt", "r")
    input_lines = finput.readlines()
    output_lines = foutput.readlines()
    target_lines = ftarget.readlines()
    finput.close()
    foutput.close()
    ftarget.close()

    # Use dictionaries to store tracks
    input_data = {'1': [], '2': [], '3': [], '4': []}
    output_data = {'1': [], '2': [], '3': [], '4': []}
    target_data = {'1': [], '2': [], '3': [], '4': []}
    for i in range(len(input_lines)):
        input_line = input_lines[i].strip()
        output_line = output_lines[i].strip()
        target_line = target_lines[i].strip()
        if input_line.endswith('INST=1 BAR_START [MASK]'):
            input_data['1'] += [input_line[:-6] + 'BAR_END']
            output_data['1'] += [output_line]
            target_data['1'] += [target_line]
        elif input_line.endswith('INST=2 BAR_START [MASK]'):
            input_data['2'] += [input_line[:-6] + 'BAR_END']
            output_data['2'] += [output_line]
            target_data['2'] += [target_line]
        elif input_line.endswith('INST=3 BAR_START [MASK]'):
            input_data['3'] += [input_line[:-6] + 'BAR_END']
            output_data['3'] += [output_line]
            target_data['3'] += [target_line]
        elif input_line.endswith('INST=4 BAR_START [MASK]'):
            input_data['4'] += [input_line[:-6] + 'BAR_END']
            output_data['4'] += [output_line]
            target_data['4'] += [target_line]
    return input_data, output_data, target_data

In [ ]:
input, output, target = process_files()
input

In [ ]:
len(input['4'])

31

In [ ]:
render_token_sequence(input['4'][1])

In [ ]:
output['4'][1]

In [ ]:
tmp = 'INST=2 BAR_START NOTE_ON=69 TIME_DELTA=12.0 NOTE_OFF=69 BAR_END BAR_START NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 NOTE_ON=69 TIME_DELTA=4.0 NOTE_OFF=69 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 BAR_END BAR_START NOTE_ON=66 TIME_DELTA=4.0 NOTE_OFF=66 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 NOTE_ON=65 TIME_DELTA=4.0 NOTE_OFF=65 NOTE_ON=65 TIME_DELTA=4.0 NOTE_OFF=65 BAR_END BAR_START NOTE_ON=66 TIME_DELTA=4.0 NOTE_OFF=66 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 NOTE_ON=69 TIME_DELTA=4.0 NOTE_OFF=69 NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 BAR_END INST=1 BAR_START NOTE_ON=73 TIME_DELTA=12.0 NOTE_OFF=73 BAR_END BAR_START NOTE_ON=71 TIME_DELTA=4.0 NOTE_OFF=71 NOTE_ON=71 TIME_DELTA=4.0 NOTE_OFF=71 NOTE_ON=73 TIME_DELTA=4.0 NOTE_OFF=73 NOTE_ON=71 TIME_DELTA=4.0 NOTE_OFF=71 BAR_END BAR_START NOTE_ON=69 TIME_DELTA=4.0 NOTE_OFF=69 NOTE_ON=71 TIME_DELTA=4.0 NOTE_OFF=71 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 NOTE_ON=68 TIME_DELTA=4.0 NOTE_OFF=68 BAR_END BAR_START NOTE_ON=69 TIME_DELTA=4.0 NOTE_OFF=69 NOTE_ON=71 TIME_DELTA=4.0 NOTE_OFF=71 NOTE_ON=73 TIME_DELTA=4.0 NOTE_OFF=73 NOTE_ON=71 TIME_DELTA=2.0 NOTE_OFF=71 NOTE_ON=73 TIME_DELTA=2.0 NOTE_OFF=73 BAR_END INST=3 BAR_START NOTE_ON=64 TIME_DELTA=12.0 NOTE_OFF=64 BAR_END BAR_START NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=64 TIME_DELTA=2.0 NOTE_OFF=64 NOTE_ON=62 TIME_DELTA=2.0 NOTE_OFF=62 BAR_END BAR_START NOTE_ON=61 TIME_DELTA=4.0 NOTE_OFF=61 NOTE_ON=62 TIME_DELTA=4.0 NOTE_OFF=62 NOTE_ON=61 TIME_DELTA=4.0 NOTE_OFF=61 NOTE_ON=61 TIME_DELTA=4.0 NOTE_OFF=61 BAR_END BAR_START NOTE_ON=61 TIME_DELTA=4.0 NOTE_OFF=61 NOTE_ON=64 TIME_DELTA=4.0 NOTE_OFF=64 NOTE_ON=64 TIME_DELTA=8.0 NOTE_OFF=64 BAR_END INST=4 BAR_START NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 NOTE_ON=61 TIME_DELTA=2.0 NOTE_OFF=61 NOTE_ON=59 TIME_DELTA=4.0 NOTE_OFF=59 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 BAR_END BAR_START NOTE_ON=53 TIME_DELTA=2.0 NOTE_OFF=53 NOTE_ON=50 TIME_DELTA=4.0 NOTE_OFF=50 NOTE_ON=47 TIME_DELTA=4.0 NOTE_OFF=47 NOTE_ON=45 TIME_DELTA=4.0 NOTE_OFF=45 NOTE_ON=50 TIME_DELTA=2.0 NOTE_OFF=50 BAR_END BAR_START NOTE_ON=54 TIME_DELTA=2.0 NOTE_OFF=54 NOTE_ON=52 TIME_DELTA=4.0 NOTE_OFF=52 NOTE_ON=54 TIME_DELTA=4.0 NOTE_OFF=54 NOTE_ON=52 TIME_DELTA=4.0 NOTE_OFF=52 NOTE_ON=57 TIME_DELTA=2.0 NOTE_OFF=57 BAR_END BAR_START NOTE_ON=54 TIME_DELTA=4.0 NOTE_OFF=54 NOTE_ON=54 TIME_DELTA=4.0 NOTE_OFF=54 NOTE_ON=56 TIME_DELTA=4.0 NOTE_OFF=56 NOTE_ON=57 TIME_DELTA=4.0 NOTE_OFF=57 BAR_END'

In [ ]:
render_token_sequence(tmp)

In [ ]:
render_token_sequence(target['4'][1])